In [56]:
import pandas as pd
import sqlite3
import numpy as np

In [57]:
feature_id = 23
negative_rt_delta_tolerance = -0.25
positive_rt_delta_tolerance = 0.25
negative_scan_delta_tolerance = -4.0
positive_scan_delta_tolerance = 0.1
maximum_number_of_features = 1
maximum_number_of_peaks_per_feature = 1

In [58]:
DB_NAME = '/home/ubuntu/UPS2_allion/UPS2_allion-features-1-1097.sqlite'
CONV_DB_NAME = '/home/ubuntu/UPS2_allion/UPS2_allion.sqlite'

In [73]:
db_conn = sqlite3.connect(DB_NAME)
peak_correlation_df = pd.read_sql_query("select * from peak_correlation where feature_id=={} and rt_distance >= {} and rt_distance <= {} and scan_distance >= {} and scan_distance <= {} order by rt_distance ASC limit {}".format(feature_id, negative_rt_delta_tolerance, positive_rt_delta_tolerance, negative_scan_delta_tolerance, positive_scan_delta_tolerance, maximum_number_of_peaks_per_feature), db_conn)
db_conn.close()

In [74]:
peak_correlation_df["feature_id-ms2_peak_id"] = peak_correlation_df.feature_id.astype(str) + '-' + peak_correlation_df.ms2_peak_id.astype(str)

In [75]:
peak_correlation_df.head()

,feature_id,base_peak_id,ms1_scan_centroid,ms1_rt_centroid,ms2_peak_id,ms2_scan_centroid,ms2_rt_centroid,scan_distance,rt_distance,correlation,feature_id-ms2_peak_id
0,23,2,110.144302,1151.36315,835,111.295575,1151.612901,-1.151273,-0.249751,0.0,23-835


In [76]:
tuple(peak_correlation_df["feature_id-ms2_peak_id"])

('23-835',)

In [77]:
peak_correlation_df = peak_correlation_df.append(peak_correlation_df)

In [84]:
len(peak_correlation_df)

2

In [78]:
peak_correlation_df

,feature_id,base_peak_id,ms1_scan_centroid,ms1_rt_centroid,ms2_peak_id,ms2_scan_centroid,ms2_rt_centroid,scan_distance,rt_distance,correlation,feature_id-ms2_peak_id
0,23,2,110.144302,1151.36315,835,111.295575,1151.612901,-1.151273,-0.249751,0.0,23-835
0,23,2,110.144302,1151.36315,835,111.295575,1151.612901,-1.151273,-0.249751,0.0,23-835


In [79]:
db_conn = sqlite3.connect(DB_NAME)
ms2_peaks_df = pd.read_sql_query("select feature_id,peak_id,centroid_mz,intensity from ms2_peaks where feature_id || '-' || peak_id in {}".format(tuple(peak_correlation_df["feature_id-ms2_peak_id"])), db_conn)
db_conn.close()

In [80]:
ms2_peaks_df.head()

,feature_id,peak_id,centroid_mz,intensity
0,23,835,627.372,73883


In [81]:
df = pd.merge(ms2_peaks_df, peak_correlation_df, left_on=['feature_id','peak_id'], right_on=['feature_id','ms2_peak_id'])

In [82]:
df.drop(['peak_id','correlation','feature_id-ms2_peak_id'], inplace=True, axis=1)

In [83]:
df.rename(columns={'intensity': 'ms2_peak_intensity', 'rt_distance': 'rt_delta', 'scan_distance': 'scan_delta', 'centroid_mz': 'ms2_peak_centroid_mz'}, inplace=True)